# Test agent

In [5]:
from pydantic import BaseModel
from pydantic_ai import Agent
from pydantic import Field
from pydantic_ai import RunContext
from dotenv import load_dotenv
from typing import Optional
import duckdb

load_dotenv()

system_prompt = """
You are a real estate agent and you need to help the customer find the best property for them.
For any property search, you MUST use the `search_properties` tool.
Do not invent information about the properties. If the search does not return results, inform the user in a friendly way.

You respond in the same language as the user's input.
"""

class UserInput(BaseModel):
    class Config:
        arbitrary_types_allowed = True
        
    connection: duckdb.DuckDBPyConnection = Field(..., description="Connection with the database")

class RealStateAgentOutput(BaseModel):
    response: str = Field(..., description="Agent response")
    properties: Optional[str] = Field(None, description="Properties found by the `search_properties` tool")

real_state_agent = Agent(
    "groq:llama-3.3-70b-versatile",
    deps_type=UserInput,
    system_prompt=system_prompt,
    output_type=RealStateAgentOutput
)

@real_state_agent.tool(require_parameter_descriptions=True)
async def search_properties(
    ctx: RunContext[UserInput],
    preco_min: float = None,
    preco_max: float = None,
    tamanho_min: float = None,
    tamanho_max: float = None,
    n_quartos: int = None,
    n_banheiros: int = None,
    n_garagem: int = None,
    rua: str = None,
    bairro: str = None,
    cidade: str = None,
) -> str:
    """
    Searches for properties in the database based on the provided filters.

    Args:
        preco_min: Minimum price of the property.
        preco_max: Maximum price of the property.
        tamanho_min: Minimum size of the property.
        tamanho_max: Maximum size of the property.
        n_quartos: Number of bedrooms.
        n_banheiros: Number of bathrooms.
        n_garagem: Number of garage spaces.
        rua: Street name (partial match).
        bairro: Neighborhood name (partial match).
        cidade: City name (partial match).

    Returns:
        A markdown table with the properties found, or an empty table if no properties match.
    """
    base_query = "SELECT id, preco, tamanho, cidade, bairro, rua, n_quartos, n_banheiros, n_garagem FROM propriedades"
    filters = []
    params = {}

    if preco_min is not None:
        filters.append(f"preco >= {preco_min}")
        params["preco_min"] = preco_min
    if preco_max is not None:
        filters.append(f"preco <= {preco_max}")
        params["preco_max"] = preco_max
    if tamanho_min is not None:
        filters.append(f"tamanho >= {tamanho_min}")
        params["tamanho_min"] = tamanho_min
    if tamanho_max is not None:
        filters.append(f"tamanho <= {tamanho_max}")
        params["tamanho_max"] = tamanho_max
    if n_quartos is not None:
        filters.append(f"n_quartos = {n_quartos}")
        params["n_quartos"] = n_quartos
    if n_banheiros is not None:
        filters.append(f"n_banheiros = {n_banheiros}")
        params["n_banheiros"] = n_banheiros
    if n_garagem is not None:
        filters.append(f"n_garagem = {n_garagem}")
        params["n_garagem"] = n_garagem
    if rua is not None:
        filters.append(f"rua ILIKE '%{rua}%'")
        params["rua"] = f"%{rua}%"
    if bairro is not None:
        filters.append(f"bairro ILIKE '%{bairro}%'")
        params["bairro"] = f"%{bairro}%"
    if cidade is not None:
        filters.append(f"cidade ILIKE '%{cidade}%'")
        params["cidade"] = f"%{cidade}%"

    if filters:
        base_query += " WHERE " + " AND ".join(filters)

    df = ctx.deps.connection.execute(base_query).fetch_df()
    return df.to_markdown(index=False)

In [6]:
connection = duckdb.connect("../db/db.duckdb")

result = await real_state_agent.run("Gostaria de um imóvel de no máximo 500mil reais, no agua verde em curitiba, 2 quartos", deps=UserInput(connection=connection))

print(result)

AgentRunResult(output=RealStateAgentOutput(response='Olá! Encontramos 6 opções de imóveis de 2 quartos no bairro Água Verde, em Curitiba, com preço máximo de R$ 500.000,00. Aqui estão os resultados da busca: \n | id                               |   preco |   tamanho | cidade   | bairro     | rua                               |   n_quartos |   n_banheiros |   n_garagem |\n|:---------------------------------|--------:|----------:|:---------|:-----------|:----------------------------------|------------:|--------------:|------------:|\n| c8e64248f1a9e66e3518454950d39545 |  460000 |        52 | curitiba | agua verde | Rua Amazonas, 383                 |           2 |             1 |           0 |\n| 077f1ace404edd106c67c285c7d11c14 |  455000 |        64 | curitiba | agua verde | Avenida República Argentina, 2534 |           2 |             3 |           0 |\n| 17f4ccd5544d36d61c98f2d0079ea74c |  469000 |        56 | curitiba | agua verde | Rua Guilherme Pugsley, 2620       |           2 | 

In [10]:
print(result.output.properties)

| id                               |   preco |   tamanho | cidade   | bairro     | rua                               |   n_quartos |   n_banheiros |   n_garagem |
|:---------------------------------|--------:|----------:|:---------|:-----------|:----------------------------------|------------:|--------------:|------------:|
| c8e64248f1a9e66e3518454950d39545 |  460000 |        52 | curitiba | agua verde | Rua Amazonas, 383                 |           2 |             1 |           0 |
| 077f1ace404edd106c67c285c7d11c14 |  455000 |        64 | curitiba | agua verde | Avenida República Argentina, 2534 |           2 |             3 |           0 |
| 17f4ccd5544d36d61c98f2d0079ea74c |  469000 |        56 | curitiba | agua verde | Rua Guilherme Pugsley, 2620       |           2 |             1 |           0 |
| 6f9666d037838cb8aab548d14c451cae |  389900 |        55 | curitiba | agua verde | Rua Guilherme Pugsley, 2650       |           2 |             1 |           0 |
| 79ab864711c3cece1b3d